# Introduction to Letta using the `LocalClient` 
This notebook is a tutorial on how to use Letta's `LocalClient`. Unlike the `RESTClient` which connects to a running agents service, the `LocalClient` will run agents on your local machine, so does not require connecting to a service. 

This tutorial will cover the basics of creating an agent, interacting with an agent, and understanding the agent's state and memories. 

## Step 0: Install the `letta` package 

In [ ]:
!pip install letta

We'll also import a helper function to print out messages from agents in a nice format: 

In [ ]:
from helper import nb_print 

## Step 1: Create a `LocalClient` 


In [ ]:
from letta import LocalClient

client = LocalClient()

### Configuring client defaults 
Agents in Letta are model agnostic, so they can connect to different model backends (you can even switch model backends for an existing agents). For this tutorial, we'll set a client default config so that all agents are created with the free letta model endpoints. 

In [ ]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("letta")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("letta")) 

## Step 2: Creating an agent 

In [ ]:
agent_name = "simple_agent"

In [ ]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

### Messaging the agent 
Now we can message the agent! This agent will have memories about both itself and the human (you). When we send a message to the agent, we will get back a list of messages from the agents. 

Letta agents have some unique characteristics that allow them to have more advanced reasoning. Notice how: 
* The agent generates *inner thoughts* to think before it acts
* Messages to the user are generated via a `send_message` tool 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)
nb_print(response.messages)

## Step 3: Understanding agent state 
Agents are essentailly multi-step reasoning programs which make multiple call to an LLM. Letta manages what is passed to the context window in reach reasoning step. The context window includes: 
* The *system prompt* to define the agent's behavior 
* The set of *tools* the agent has access to 
* The agent's *core memory* (i.e. in-context memory)
* A summary of it's *archival memory* 
* A summary of it's *recall memory* 
* An in-context message queue

In this section, we'll look at the current state of the agent to understand exactly what is being passed to the context window. 

### System Prompt 
The system prompt defines the behavior of the agent. Unlike the memory, the system prompt is not editable. 

In [ ]:
print(agent_state.system)

### Tools 
The agent has access to a set of tools. Each tool is stored in a database, so it can be loaded and executed by the server. Letta also includes a set of default memory management tools, as well as the `send_message` tool to communicate with the human. 

In [ ]:
agent_state.tools

### Core memory 
The core memory is the part of memory that is places *in-context*. 

In [ ]:
memory = client.get_core_memory(agent_state.id)

In [ ]:
memory

In [ ]:
memory.get_block('human')

### Archival & Recall memory summaries
The agent also has access to external memories (stored in a database). There are two types of external memory: 
* *Archival memory*: Memories stored in a vector database that are either saved by the agent itself, or loaded in by the user
* *Recall memory*: The full conversational history of the agent

Both of these memories stores can be queried by the agent for RAG. To ensure the agent knows that these external memories stores may have relevant information, the context window contains a summary of the number of rows in both archival and recall memory. 

In [ ]:
client.get_archival_memory_summary(agent_state.id)

In [ ]:
client.get_recall_memory_summary(agent_state.id)

You can also directly query the full conversational history: 

In [ ]:
client.get_messages(agent_state.id)

## Section 4: Modifying core memory 

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
nb_print(response.messages)

In [ ]:
client.get_archival_memory(agent_state.id)[0].text

## Section 5: Modifying archival memory 

In [ ]:
client.insert_archival_memory(
    agent_state.id, 
    "Bob's loves boston terriers"
)

In [ ]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
nb_print(response.messages)